In [32]:
import csv
from collections import defaultdict

In [33]:
rows = []

with open('online_retail.csv') as retail:
    for row in csv.reader(retail):

        if len(row) != 8:
            continue
        rows.append(row)

In [34]:
labels = [el.lower() for el in rows[0]]
# print(rows[0], '\n', rows[1])
del rows[0]

In [35]:
print(len(rows))

for i, row in enumerate(rows):
    if row[0].startswith('C'):
        del rows[i]

print(len(rows))

541909
536442


In [36]:
invoice_aggregation = defaultdict(lambda : set())

for invoice in rows:
    invoice_aggregation[invoice[0]].add(invoice[2])

In [36]:
all_items = set()

for el in invoice_aggregation.values():
    all_items.add(el)
